In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cv2

import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from PIL import Image

In [2]:
base_path = "../Data/CK_CK+/CK+/"

In [3]:
neutralised_path = "NeutralisedImages/CK_CK+/CK+/" + 'cohn-kanade-images'
all_neutralisedImage_files = glob.glob(neutralised_path+'/**/*.png', recursive=True)

In [4]:
all_emotions = {0: 'neutral', 1: 'anger', 2: 'contempt', 3: 'disgust', 
                4: 'fear', 5: 'happy', 6: 'sadness', 7: 'surprise'}

In [5]:
import copy

class Image():
    
    def __init__(self, image, landmarks, emotion, name):
        self.image = image
        self.original_image = copy.deepcopy(self.image)
        self.name = name
        self.emotion = emotion
        self.landmarks = landmarks
        self.neutralised_image = np.array([])
        self.hash = hash(self)
    
    def getEmotion(self):
        return self.emotion
    
    def getLandmarks(self):
        return self.landmarks
    
    def setNeutralisedImage(self, image):
        self.neutralised_image = image
        
    def getNeutralisedImage(self):
        return self.neutralised_image
        
    def getName(self):
        return self.name
        
    def getImage(self):
        return self.image
    
    def clearImage(self):
        self.image= copy.deepcopy(self.original_image)
        return self.image
    
    def getHash(self):

        return self.hash

In [6]:
face_det = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
face_det2 = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_alt2.xml")
face_det3 = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_alt.xml")
face_det4 = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_alt_tree.xml")

def detect_face(f, ld):
    ''' function to detect face, crop the image and resize it and resize the corresponding landmarks too '''
    frame = cv2.imread(f) #Open image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert image to grayscale
    face = face_det.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    face2 = face_det2.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    face3 = face_det3.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    face4 = face_det4.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
#     ld = np.array([np.array(i) for i in ld])
    crop_ld = ld
    #Go over detected faces, stop at first detected face, return empty if no face.
    if len(face) == 1:
        facefeatures = face
    elif len(face2) == 1:
        facefeatures = face2
    elif len(face3) == 1:
        facefeatures = face3
    elif len(face4) == 1:
        facefeatures = face4
    else:
        facefeatures = ""
    for (x, y, w, h) in facefeatures:
        fw, fh = w, h
        if any(ld[:,0] > x+w):
            fw = int(max(ld[:,0]-x))
        if any(ld[:,1] > y+h):
            fh = int(max(ld[:,1]-y))
#         gray = gray[y:y+h, x:x+w] #Cut the frame to size
        gray = gray[y:y+fh+5, x:x+fw+5] #Cut the frame to size to max of landmark with padding of 5px
        ld_rescale_factor = (gray.shape[1]/crop_imsize, (gray.shape[0]/crop_imsize))
        crop_ld = np.array(list(zip(ld[:,0]-x, ld[:, 1]-y)))
    try:
        output = cv2.resize(gray, (crop_imsize, crop_imsize))
        crop_ld = crop_ld/ld_rescale_factor
        cv2.imwrite(crop_path+'/crop_'+f.split('/')[-1], output)
    except:
        output = gray

    return output, crop_ld

In [7]:
def pairPlot(im1, im2, clear_figure=False, title=''):
    if not clear_figure:
        plt.figure()
    
    fig, axes = plt.subplots(1,2)
    axes[0].imshow(im1)
    axes[1].imshow(im2)

    fig.suptitle(title, fontsize=16)
    
def getTrianglePts(triangle_points_array):
    return (tuple(triangle_points_array[0:2].astype(np.int32)), tuple(triangle_points_array[2:4].astype(np.int32)), tuple(triangle_points_array[4:6].astype(np.int32)))

def getTriangleImage(im, triangle_points_array):
    pt1, pt2, pt3 = getTrianglePts(triangle_points_array)
    triangle = np.array([pt1, pt2, pt3])
    
    triangle_im = im
    triangle_mask = np.zeros_like(im)

    triangle_mask_fill_points = np.array([
            [pt1[0], pt1[1]],
            [pt2[0], pt2[1]],
            [pt3[0], pt3[1]]
        ], np.int32)
    cv2.fillConvexPoly(triangle_mask, triangle_mask_fill_points, (255,255,255))
        
    cropped_triangle = cv2.bitwise_and(triangle_mask, triangle_im)
    return cropped_triangle

def cropByLandmarks(im,ld):
    landmarks = list(map(lambda x: tuple(x),ld))

    im_shape = im.shape
    
    rect = (0,0,im_shape[1],im_shape[0])
    subdiv = cv2.Subdiv2D(rect)
    subdiv.insert(landmarks)
    
    triangles = subdiv.getTriangleList()
    cropped_image = np.zeros_like(im) 
    for triangle in triangles:
        triangle_image = getTriangleImage(im, triangle)      
        cropped_image = cv2.add(cropped_image, triangle_image)

    return cropped_image

In [8]:
# from tqdm import tqdm

# crop_imsize = 350
# subjects = []

# with tqdm(total=len(all_neutralisedImage_files), desc="Processing entries") as pbar:

#     for neutral_image_path in all_neutralisedImage_files[:]:

#         original_image_path = "../Data/" + "/".join(neutral_image_path.split("/")[1:])
#         landmark_path = base_path + "Landmarks/" + "/".join(neutral_image_path.split("/")[4:])[:-4] + '_landmarks.txt'
#         emotion_path = base_path + "Emotion/" + "/".join(neutral_image_path.split("/")[4:-1])
        
#         emotion_files = glob.glob(emotion_path+'/*.txt')

#         if len(emotion_files) < 1:
#             pbar.update(1)
#             continue

#         with open(emotion_files[0]) as emotion_file:
#             emotion = emotion_file.read()
#             emotion = all_emotions[(int(float(emotion)))]

#         landmarks = np.loadtxt(landmark_path)
        
#         original_image, resized_landmarks = detect_face(original_image_path, landmarks)  
#         original_image = cv2.resize(original_image,(crop_imsize, crop_imsize))
#         original_image = cropByLandmarks(original_image, resized_landmarks)

#         neutralised_image = cv2.imread(neutral_image_path) #Open Neutralised image
# #         pairPlot(original_image, neutralised_image)
#         im_object = Image(original_image, landmarks, emotion, int((original_image_path.split("/")[-1])[:-4].split('_')[0][1:]))
#         im_object.setNeutralisedImage(neutralised_image)

#         subjects.append(im_object)
#         pbar.update(1)

## Half of the images do not have an emotion???!

In [9]:
# len(subjects)

# Create datasets for face recognition and exp recognition

In [10]:
# images_data = []

# for image in subjects:
#     images_data.append(np.array([image.getNeutralisedImage(), image.getImage(), image.getEmotion(), image.getName()]))

# images_df = pd.DataFrame(images_data, columns=['neutralised_image', 'original_image', 'original_emotion', 'name'])

In [11]:
# images_df.to_pickle('./joined_images_df.pkl')

In [2]:
images_df = pd.read_pickle('./joined_images_df.pkl')

In [5]:
images_df = images_df.iloc[:1000]

In [6]:
X_rec_origin, y_rec_origin = images_df[['original_image', 'neutralised_image']], images_df['name']

X_rec_origin_train, X_rec_origin_test, y_rec_origin_train, y_rec_origin_test = train_test_split(X_rec_origin, y_rec_origin, test_size=0.33, random_state=42)
X_rec_origin_train.reset_index(drop=True, inplace=True), X_rec_origin_test.reset_index(drop=True, inplace=True)
y_rec_origin_train.reset_index(drop=True, inplace=True), y_rec_origin_test.reset_index(drop=True, inplace=True)
X_rec_origin_train.shape, X_rec_origin_test.shape, y_rec_origin_train.shape, y_rec_origin_test.shape

((670, 2), (330, 2), (670,), (330,))

In [7]:
np.array(y_rec_origin_train)

array([ 54, 117,  37,  53, 111, 117,  54,  65,  55,  65,  65,  62, 127,
       116,  62,  62,  65,  52, 116,  99, 119, 128,  52, 129, 128, 127,
       117,  62, 117, 127,  62,  91, 127,  62, 127, 111,  37, 119,  62,
        55, 127,  54, 127, 127, 111,  37,  65, 129,  52,  55, 126, 127,
        55, 126, 119,  52,  65, 127,  90,  55, 119, 129,  54, 116,  91,
       129,  62,  62,  37,  90, 111, 895,  65,  65,  53,  54,  52, 110,
       129,  65,  65,  52,  55, 126,  55,  54,  54,  55,  65,  62, 111,
        37,  65,  91,  62,  96, 129, 129,  98, 128, 128,  96,  96,  62,
       128, 111, 119,  96,  52,  53, 119,  91,  99,  98,  65,  91,  53,
       127,  54,  54,  91,  55,  62, 127, 126,  53,  52,  54,  96,  62,
       129, 116,  53,  99,  62, 116, 116,  62,  65,  62,  62, 116,  52,
       116, 117,  90,  52,  52,  53,  53,  53, 116, 111,  62,  65, 129,
       129, 117, 127,  65,  52,  53,  37, 116,  62, 119,  62,  62,  55,
        52,  54, 119,  62,  52,  55,  99,  54,  62,  99,  37, 12

# Fisher Face

In [ ]:
fishface = cv2.face.FisherFaceRecognizer_create() #Initialize fisher face classifier
fishface.train(X_rec_origin_train['original_image'].to_numpy(), np.array(y_rec_origin_train))

In [ ]:
correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['original_image']):
    pred, conf = fishface.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotion Acc: ', (100*correct)/(correct + incorrect))

correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['neutralised_image']):
    pred, conf = fishface.predict(image)
    if pred == np.array(y_rec_origin_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotionless Acc: ', (100*correct)/(correct + incorrect))

In [ ]:
print(fishface.predict(X_rec_origin_test['original_image'][10]), y_rec_origin_test[10], 
      fishface.predict(X_rec_origin_test['neutralised_image'][10]), y_rec_origin_test[10])

# Eigen Face

In [ ]:
eigenface = cv2.face.EigenFaceRecognizer_create()
eigenface.train(X_train['im_neutral_crop'], np.array(y_train))
correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotion_crop']):
    pred, conf = eigenface.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotion Acc: ', (100*correct)/(correct + incorrect))

correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotionless']):
    pred, conf = eigenface.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotionless Acc: ', (100*correct)/(correct + incorrect))

In [ ]:
print(eigenface.predict(X_test['im_emotion_crop'][10]), y_test[10], 
      eigenface.predict(X_test['im_emotionless'][10]), y_test[10])

# LBHP 

In [ ]:
lbph_face = cv2.face.LBPHFaceRecognizer_create()
lbph_face.train(X_train['im_neutral_crop'], np.array(y_train))
correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotion_crop']):
    pred, conf = lbph_face.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotion Acc: ', (100*correct)/(correct + incorrect))

correct = 0
incorrect = 0
for cnt, image in enumerate(X_test['im_emotionless']):
    pred, conf = lbph_face.predict(image)
    if pred == np.array(y_test)[cnt]:
        correct += 1
    else:
        incorrect += 1
print('Emotionless Acc: ', (100*correct)/(correct + incorrect))

In [ ]:
print(lbph_face.predict(X_test['im_emotion_crop'][10]), y_test[10], 
      lbph_face.predict(X_test['im_emotionless'][10]), y_test[10])